## accesso al daily firstrate

In [36]:
from revelation.data.loading import (
    Catalog,
    firstrate_dirname,
    firstrate_filename,
    CSVPresets,
)
import pandas as pd

catalog = Catalog()

ohlc_d: pd.DataFrame = catalog.get_csv(
    catalog._raw_directory
    / "csv/firstrate"
    / firstrate_dirname()
    / firstrate_filename(),
    preset=CSVPresets.FIRSTRATE,
)

In [37]:
# print(ohlc_d.info())
print("is_unique", ohlc_d.index.is_unique)
print("is_sorted", ohlc_d.index.is_monotonic_increasing)

is_unique True
is_sorted True


## accesso ai dati tradingview

In [38]:
import vectorbtpro as vbt
from revelation.time_utils import STANDARD_TIMEZONE
from dotenv import load_dotenv

load_dotenv()
import os

tradingview_client = vbt.TVData.resolve_client(
    # username=os.getenv("TRADINGVIEW_USERNAME"),
    # password=os.getenv("TRADINGVIEW_PASSWORD"),
    auth_token=os.getenv("TRADINGVIEW_AUTH_TOKEN"),
)

In [39]:
data = vbt.TVData.fetch_symbol(
    "CME:6EH2024", timeframe="1 hour", client=tradingview_client
)
data[0]

,Open,High,Low,Close,Volume
datetime,,,,,
2021-02-05 20:00:00+00:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 20:00:00+00:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 20:00:00+00:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 20:00:00+00:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 20:00:00+00:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 10:00:00+00:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 11:00:00+00:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 12:00:00+00:00,1.09025,1.09045,1.08915,1.08915,101.0


In [40]:
df: pd.DataFrame = data[0].copy()
df.index = df.index.tz_convert(STANDARD_TIMEZONE)
df.columns = df.columns.str.lower()
df

,open,high,low,close,volume
datetime,,,,,
2021-02-05 15:00:00-05:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 15:00:00-05:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 15:00:00-05:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 15:00:00-05:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 15:00:00-05:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 06:00:00-04:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 07:00:00-04:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 08:00:00-04:00,1.09025,1.09045,1.08915,1.08915,101.0


In [41]:
from revelation.visualization import hvplot_ohlc
import holoviews as hv

chart = hvplot_ohlc(df)
chart

:Overlay
   .Segments.I   :Segments   [datetime,low,datetime,high]   (open,high,low,close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)

In [42]:
import pandas as pd
import holoviews as hv


def hvplot_labels(
    labels: pd.Series,
    price_df: pd.DataFrame | None = None,
    y_from: str = "close",
    y_offset: float = 0.0,
    **label_opts,
):
    """
    Convert a Series of events into an hv.Labels element.

    Parameters
    ----------
    labels : pd.Series
        Series whose **index** are the datetime points to annotate and whose
        **values** are the strings you want displayed.  Any NaN/empty entries
        are ignored.
    price_df : pd.DataFrame, optional
        The OHLC dataframe you plotted.  If given, the function will position
        each label at `price_df.loc[t, y_from] + y_offset`.
        If it is *None* and the Series values are numeric, those numeric
        values are used as the y-coordinate.
    y_from : str, default "close"
        Column of `price_df` to use for the y-coordinate (e.g. "high", "low").
    y_offset : float, default 0
        Constant offset added to the y-coordinate—handy for keeping the text
        clear of the candle body/wick.
    **label_opts
        Extra arguments forwarded to `hv.Labels.opts` (text_color, text_align,
        text_font_size, etc.).

    Returns
    -------
    hv.Labels
    """
    # keep only rows that actually have something to show
    s = labels.dropna()

    if price_df is not None:
        y = price_df.loc[s.index, y_from] + y_offset
    else:
        # if the Series itself is numeric, use those numbers;
        # otherwise fall back to a flat baseline at zero
        y = (
            s.astype(float)
            if pd.api.types.is_numeric_dtype(s)
            else pd.Series([0] * len(s), index=s.index)
        )

    df_lbl = pd.DataFrame({"x": s.index, "y": y, "text": s.astype(str).values})

    return hv.Labels(df_lbl).opts(**label_opts)


events = pd.Series(
    ["FOMC", "NFP", "ECB"],
    index=[df.index[-1], df.index[-2], df.index[-3]],
    name="Event",
)
events.index = events.index.tz_convert("America/New_York")

labels = hvplot_labels(events, df)
# Combinazione
(chart * labels)

TypeError: Cannot compare tz-naive and tz-aware timestamps

:Overlay
   .Segments.I   :Segments   [datetime,low,datetime,high]   (open,high,low,close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)
   .Labels.I     :Labels   [x,y]   (text)